In [17]:
import os
import sys
import time  # 強制等待
import requests
import pandas as pd  # 載入 pandas
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options  # 設定 driver 的行為
from selenium.webdriver.support.ui import Select  # 選擇＂下拉式選單＂
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # 載入常見錯誤
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  # 更改載入策略
from selenium.webdriver.support.ui import WebDriverWait  # 等待機制
from selenium.webdriver.support import expected_conditions as EC  # 預期事件
from selenium.webdriver.common.by import By  # 找尋元素的方法

In [18]:
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = os.environ.get('GOOGLE_CHROME_BIN')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

# chrome_options.add_argument('disable-dev-shm-usage')
# chrome_options.add_argument('--no-sandbox')
chrome_capabilities = DesiredCapabilities.CHROME
chrome_capabilities['pageLoadStrategy'] = 'eager'  # 頁面加載策略：HTML 解析成 DOM
def get_chrome():
    return webdriver.Chrome(options=chrome_options,
                            desired_capabilities=chrome_capabilities)

In [32]:

def wait_for_element_clickable(driver, element_position, waiting_time=5, by=By.LINK_TEXT):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    try:
        node_off()
        element = WebDriverWait(driver, waiting_time).until(EC.element_to_be_clickable((by, element_position)))
    except Exception as e:
        alert_exception_report(function, e)
        return False
    else:
        return element

def accurately_find_table_and_read_it(driver, table_position, table_index=0):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    try:
        node_off()
        if not wait_for_element_present(driver, table_position):
            alert_exception_report(function, 'not found table')
            return
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_innerHTML = soup.select(table_position)[table_index]
        tgt = pd.read_html(str(table_innerHTML), encoding='utf-8')[0]
        # tgt['圖書館'], tgt['連結'] = org, driver.current_url
    except Exception as e:
        alert_exception_report(function, e)
        return
    else:
        return tgt

def get_all_tgt_urls(driver, link_text):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    
    tgt_urls = []

    anchors = driver.find_elements_by_link_text(link_text)
    for anchor in anchors:
        tgt_urls.append(anchor.get_attribute('href'))
    
    return tgt_urls

In [19]:
def get_employee_data(driver):
    # Wait for the pop-up to appear
    WebDriverWait(driver, 5).until(EC.presence_of_elements_located((By.ID, "tableName")))
    
    # Get the HTML content of the pop-up
    detail_html = driver.page_source
    
    # Parse the HTKL content with BeautifulSoup
    detail_soup = BeautifulSoup(detail_html, "html.parser")
    
    # Find the table containing the data
    table = detail_soup.find("table", {"class": "hasBorder"})
    
    # Extract data from the table
    data = []
    for row in table.find_all("tr"):
        cells = row.find_all("td")
        if len(cells) > 1:
            data.append([cell.text.strip() for cell in cells])
    
    return data

In [42]:
# Set up Selenium WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://mops.twse.com.tw/mops/web/t100sb13'

driver.get(url)

year_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "year")))
year_input.clear()
year_input.send_keys("111")

search_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[3]/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]/div/div/input")))
search_button.click()

# Find the table containing the data
table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[5]/div/form/center/table")))

# Extract data from the table
data = []
for row in table.find_elements(By.TAG_NAME, "tr"):
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) > 1:
        # Extract '公司代號'
        代號 = cells[0].text.strip()
        
        # Click on DETAIL button
        detail_button = cells[-1].find_element(By.ID, "DETAIL")
        detail_button.click()
        
        detail_data = get_employee_data(driver)
        
        data.append((代號, detail_data))
        
        close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(By.XPATH, "//input[@value='關閉']"))
        close_button.click()
        
driver.quit()

df = pd.DataFrame(data, columns=['代號', '詳細資料'])

AttributeError: module 'selenium.webdriver.support.expected_conditions' has no attribute 'presence_of_elements_located'

In [41]:
df

,代號,詳細資料


In [ ]:
url = 'https://mops.twse.com.tw/mops/web/t100sb13'
response = request.get(url)